# Data Cleaning in Python

This dataset was obtained through web scrapping of OpenRice website: https://sg.openrice.com/en/singapore

### Importing libraries

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
sb.set()

### Checking the dataset

After importing the relevant libraries, we'll need to check the dataset for missing values

In [2]:
projectD = pd.read_csv('features.csv')
projectD.head()

,name,street_address,price,cuisine,rating,latitude,longitude
0,1-V:U,"The Outpost Hotel Sentosa, 10 Artillery Avenue...",$31 - $50,Asian Variety,NaN,1.252299,103.820211
1,10 At Claymore,"Pan Pacific Orchard, 10 Claymore Road Level 2",$51 - $80,Multi-Cuisine,4.0,1.307401,103.829904
2,10 SCOTTS,"Grand Hyatt Singapore, 10 Scotts Road Lobby Level",$31 - $50,NaN,3.5,1.306345,103.833283
3,100g Korean BBQ,93 Amoy Street,$21 - $30,Korean,3.5,1.281299,103.847092
4,10th Chocolate Street,"GSM Buidling, 141 Middle Road",$11 - $20,Belgian,NaN,1.299438,103.852403


In [3]:
projectD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3998 entries, 0 to 3997
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            3998 non-null   object 
 1   street_address  3998 non-null   object 
 2   price           3992 non-null   object 
 3   cuisine         2874 non-null   object 
 4   rating          2287 non-null   float64
 5   latitude        3965 non-null   float64
 6   longitude       3965 non-null   float64
dtypes: float64(3), object(4)
memory usage: 218.8+ KB


### Dealing with duplicates and assigning unique IDs

Currently, restaurants are identified by their name, which is a string value. Do we have duplicate names?

In [4]:
dup_names = projectD[projectD['name'].duplicated(keep=False)]
print("There are {} restaurant names which have duplicates.".format(len(dup_names)
                                                                    - dup_names['name'].duplicated().sum()))
dup_names.head()

There are 383 restaurant names which have duplicates.


,name,street_address,price,cuisine,rating,latitude,longitude
6,126 Wen Dou Sek,126 Sims Avenue,$11 - $20,Cantonese/Hong Kong,4.0,1.314219,103.878429
7,126 Wen Dou Sek,1086 Serangoon Road,Under $10,Cantonese/Hong Kong,NaN,1.323609,103.864960
12,136 Hong Kong Street Fish Head Steamboat,291 South Bridge Road,$11 - $20,"Teochew, Chinese",3.5,1.281415,103.845043
13,136 Hong Kong Street Fish Head Steamboat,Blk 19 Lorong 7 Toa Payoh #01-264,$11 - $20,Cantonese/Hong Kong,NaN,1.335330,103.856453
30,328 Katong Laksa,"Queensway Shopping Centre, 1 Queensway #01-60",Under $10,Peranakan / Nonya,3.5,1.287477,103.803400


It looks like the duplicated names are of restaurants with multiple branches in different locations. Is the combination of name and street address at least unique?

In [5]:
print("There are {} rows with duped names and street addresses."\
      .format(projectD[['name', 'street_address']].duplicated(keep=False).sum()))

There are 30 rows with duped names and street addresses.


Sadly, the answer is no... Let's extract and look at the rows with both duplicated names and addresses.

In [6]:
full_dup = projectD[projectD[['name', 'street_address']].duplicated(keep=False)]
full_dup.head()

,name,street_address,price,cuisine,rating,latitude,longitude
285,Basilico,"Regent Singapore, 1 Cuscaden Road",$31 - $50,"Italian, Middle Eastern/Mediterranean, European",4.0,1.304976,103.825429
286,Basilico,"Regent Singapore, 1 Cuscaden Road",$51 - $80,Italian,4.0,1.304976,103.825429
311,Beast &amp; Butterflies,90 Robertson Quay,$31 - $50,"Western Variety, Asian Variety",NaN,1.290326,103.836860
312,Beast &amp; Butterflies,90 Robertson Quay,$51 - $80,Multi-Cuisine,NaN,1.290326,103.836860
373,Bistro du Vin,"Shaw Centre, 1 Scotts Road #01-14",$31 - $50,"French, Middle Eastern/Mediterranean, European",4.0,1.306359,103.831646


It looks like these rows correspond to the same restaurants in the same places, but with different cuisines. Let's reduce each such group of duplicated rows to one, by concatenating the cuisines. For other columns we will arbitrarily pick that of the first row in the group.

In [7]:
unduped = full_dup.groupby(by=['name', 'street_address']).aggregate({
    'price': 'first',
    'cuisine': lambda x: x.drop_duplicates().str.cat(sep=', '),
    'rating': 'first',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()
unduped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            15 non-null     object 
 1   street_address  15 non-null     object 
 2   price           15 non-null     object 
 3   cuisine         15 non-null     object 
 4   rating          10 non-null     float64
 5   latitude        15 non-null     float64
 6   longitude       15 non-null     float64
dtypes: float64(3), object(4)
memory usage: 968.0+ bytes


Now let's remove the duplicated restaurants from our original dataframe and append our reduced version in their place. 

In [8]:
projectD = projectD.drop_duplicates(subset=['name', 'street_address'], keep=False).append(unduped)
projectD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 14
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            3983 non-null   object 
 1   street_address  3983 non-null   object 
 2   price           3977 non-null   object 
 3   cuisine         2861 non-null   object 
 4   rating          2278 non-null   float64
 5   latitude        3950 non-null   float64
 6   longitude       3950 non-null   float64
dtypes: float64(3), object(4)
memory usage: 248.9+ KB


Success! Now we can be confident that every row in our dataset corresponds to one restaurant location. Our unique identifier being two string columns is awkward, however; let's assign each row a unique index number which will be preserved despite the Pandas operations we do.

In [9]:
projectD.insert(0, 'id', projectD.index)
projectD.head()

,id,name,street_address,price,cuisine,rating,latitude,longitude
0,0,1-V:U,"The Outpost Hotel Sentosa, 10 Artillery Avenue...",$31 - $50,Asian Variety,NaN,1.252299,103.820211
1,1,10 At Claymore,"Pan Pacific Orchard, 10 Claymore Road Level 2",$51 - $80,Multi-Cuisine,4.0,1.307401,103.829904
2,2,10 SCOTTS,"Grand Hyatt Singapore, 10 Scotts Road Lobby Level",$31 - $50,NaN,3.5,1.306345,103.833283
3,3,100g Korean BBQ,93 Amoy Street,$21 - $30,Korean,3.5,1.281299,103.847092
4,4,10th Chocolate Street,"GSM Buidling, 141 Middle Road",$11 - $20,Belgian,NaN,1.299438,103.852403


### Dealing with missing values

The following problems remain with the dataset.

> **price** : missing values & currently in string format eg.\\$11-\\$30   
> **cuisine** : missing values & contains multiple values in a cell separated by "/" and "," eg. Italian, Japanese   
> **rating** : missing values & does not contain ratings lower than 2   
> **latitude** & **longitude** : missing values   

We begin by dropping the rows containing missing values in prices, latitude or longitude.

In [10]:
projectD = projectD.dropna(subset=['price','latitude','longitude'])
projectD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3944 entries, 0 to 14
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              3944 non-null   int64  
 1   name            3944 non-null   object 
 2   street_address  3944 non-null   object 
 3   price           3944 non-null   object 
 4   cuisine         2833 non-null   object 
 5   rating          2259 non-null   float64
 6   latitude        3944 non-null   float64
 7   longitude       3944 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 277.3+ KB


### Filling in missing values of "Rating"

We decided to fill the missing values with the ***mode* of ratings**, as that is the measure of central tendency which makes sense with discrete categories of data.

In [11]:
rate = projectD['rating']
rate_mode = rate.mode()[0]
print("The mode of rating is {}/5.".format(rate_mode))

The mode of rating is 3.5/5.


In [12]:
rate = rate.fillna(0)
rate = rate.replace(0, rate_mode)
projectD['rating'] = rate
projectD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3944 entries, 0 to 14
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              3944 non-null   int64  
 1   name            3944 non-null   object 
 2   street_address  3944 non-null   object 
 3   price           3944 non-null   object 
 4   cuisine         2833 non-null   object 
 5   rating          3944 non-null   float64
 6   latitude        3944 non-null   float64
 7   longitude       3944 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 277.3+ KB


---

### Cleaning "Price"

Price contains string values in the format \\$11-\\$30 and "Under \\$10".

We want to **treat price as numerical data**, as some price categories are meaningfully "closer" to specific other price categories, and we want our clustering algorithm to take this into account. For example, "Under 11" is close to "11-20" and far from "31-50".

Steps taken to clean price

1. Replace all "Under \\$11" with \\$0 - $10   
2. Split price into LowPrice and HighPrice   
3. String slice the \\$ out and convert value to integer   
4. Calculate the middle of LowPrice and HighPrice   
5. Make new column in dataframe with the new calculated price value

In [13]:
projectD['price'] = np.where(projectD['price'].str.contains("-"), projectD['price'], '$0 - $9')
projectD.head(6)

,id,name,street_address,price,cuisine,rating,latitude,longitude
0,0,1-V:U,"The Outpost Hotel Sentosa, 10 Artillery Avenue...",$31 - $50,Asian Variety,3.5,1.252299,103.820211
1,1,10 At Claymore,"Pan Pacific Orchard, 10 Claymore Road Level 2",$51 - $80,Multi-Cuisine,4.0,1.307401,103.829904
2,2,10 SCOTTS,"Grand Hyatt Singapore, 10 Scotts Road Lobby Level",$31 - $50,NaN,3.5,1.306345,103.833283
3,3,100g Korean BBQ,93 Amoy Street,$21 - $30,Korean,3.5,1.281299,103.847092
4,4,10th Chocolate Street,"GSM Buidling, 141 Middle Road",$11 - $20,Belgian,3.5,1.299438,103.852403
5,5,123 Seafood,"Chinatown Complex Market and Food Centre, 335 ...",$0 - $9,"Singaporean, Chinese",3.5,1.282360,103.843546


In [14]:
priceData = projectD['price'].str.split(" - ",expand=True)
LowPrice = priceData[0].str.slice(start=1)
LowPrice = LowPrice.astype('int')
HighPrice = priceData[1]
HighPrice = HighPrice.str.slice(start=1)
HighPrice = HighPrice.fillna(10)
HighPrice = HighPrice.astype('int')
projectD['price_mid'] = (HighPrice + LowPrice).div(2)
projectD.head(6)

,id,name,street_address,price,cuisine,rating,latitude,longitude,price_mid
0,0,1-V:U,"The Outpost Hotel Sentosa, 10 Artillery Avenue...",$31 - $50,Asian Variety,3.5,1.252299,103.820211,40.5
1,1,10 At Claymore,"Pan Pacific Orchard, 10 Claymore Road Level 2",$51 - $80,Multi-Cuisine,4.0,1.307401,103.829904,65.5
2,2,10 SCOTTS,"Grand Hyatt Singapore, 10 Scotts Road Lobby Level",$31 - $50,NaN,3.5,1.306345,103.833283,40.5
3,3,100g Korean BBQ,93 Amoy Street,$21 - $30,Korean,3.5,1.281299,103.847092,25.5
4,4,10th Chocolate Street,"GSM Buidling, 141 Middle Road",$11 - $20,Belgian,3.5,1.299438,103.852403,15.5
5,5,123 Seafood,"Chinatown Complex Market and Food Centre, 335 ...",$0 - $9,"Singaporean, Chinese",3.5,1.282360,103.843546,4.5


---

### Cleaning "Cuisine"

Each restaurant may have multiple cuisines attached to it; the categories are not mutually exclusive, so we will have to think about how to deal with that before we run clustering.

Before that we'll parse the data, which is in the format of cuisines separated by slashes and commas. We will make a list with each individual cuisine and make a new column which is a list of indices into our cuisines list.

We will drop the missing values of cuisine before doing this.


In [15]:
projectD = projectD.dropna(subset=['cuisine'])
projectD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2833 entries, 0 to 14
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2833 non-null   int64  
 1   name            2833 non-null   object 
 2   street_address  2833 non-null   object 
 3   price           2833 non-null   object 
 4   cuisine         2833 non-null   object 
 5   rating          2833 non-null   float64
 6   latitude        2833 non-null   float64
 7   longitude       2833 non-null   float64
 8   price_mid       2833 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 221.3+ KB


In [16]:
# What cuisine categories do we have?
cuisines = []
for idx, row in projectD.iterrows():
    vals = row['cuisine']
    if vals == vals: #test for NaNs
        vals = vals.replace('/', ', ')
        vals = vals.split(', ')
        
        for val in vals:
            if val not in cuisines:
                cuisines.append(val)
                
print(cuisines)

['Asian Variety', 'Multi-Cuisine', 'Korean', 'Belgian', 'Singaporean', 'Chinese', 'Cantonese', 'Hong Kong', 'Teochew', 'Malay', 'Middle Eastern', 'Mediterranean', 'Thai', 'Peranakan ', ' Nonya', 'Singaporean Chinese', 'American', 'Taiwanese', 'French', 'Italian', 'Indian', 'Fusion', 'Hainanese', 'Singaporean Western', 'Japanese', 'Malaysian', 'Fujian', 'Heng Hwa', 'European', 'Australian ', ' New Zealand', 'English', 'Shanghainese', 'Indonesian', 'Spanish', 'Mexican', 'Portuguese', 'Sichuan', 'German', 'Latin American', 'South American', 'Swiss', 'Western Variety', 'Vietnamese', 'Beijing', 'Russian', 'Foochow', 'Hakka', 'Caribbean', 'Filipino', 'Irish', 'Dong Bei']


In [17]:
projectD['cuisine_cats'] = projectD.apply(lambda x: [] if x[4] != x[4]
                                          else [cuisines.index(i) for i in x[4].replace(', ', '/').split('/')], axis=1)
projectD.head()

,id,name,street_address,price,cuisine,rating,latitude,longitude,price_mid,cuisine_cats
0,0,1-V:U,"The Outpost Hotel Sentosa, 10 Artillery Avenue...",$31 - $50,Asian Variety,3.5,1.252299,103.820211,40.5,[0]
1,1,10 At Claymore,"Pan Pacific Orchard, 10 Claymore Road Level 2",$51 - $80,Multi-Cuisine,4.0,1.307401,103.829904,65.5,[1]
3,3,100g Korean BBQ,93 Amoy Street,$21 - $30,Korean,3.5,1.281299,103.847092,25.5,[2]
4,4,10th Chocolate Street,"GSM Buidling, 141 Middle Road",$11 - $20,Belgian,3.5,1.299438,103.852403,15.5,[3]
5,5,123 Seafood,"Chinatown Complex Market and Food Centre, 335 ...",$0 - $9,"Singaporean, Chinese",3.5,1.282360,103.843546,4.5,"[4, 5]"


Some observations of ours are that there are 52 distinct cuisines, and that a restaurant has at most 6 cuisines.

In [18]:
print("Cuisines: {}".format(len(cuisines)))
print("The most cuisines in a restaurant: {}".format(projectD['cuisine_cats'].apply(len).max()))

Cuisines: 52
The most cuisines in a restaurant: 6


We will turn the rows with more than one cuisine into multiple rows, each with one cuisine, in order to make it easier to do clustering/train models later on on our data. We will save this duplicated data as a separate dataframe.

In [19]:
dupedD = pd.DataFrame(projectD)
dupedD['duplicate'] = dupedD.apply(lambda x: 1 if len(x[9])>1 else 0, axis=1)
dupedD = dupedD.explode('cuisine_cats').rename(columns={'cuisine_cats':'cuisine_cat'})
dupedD.head()

,id,name,street_address,price,cuisine,rating,latitude,longitude,price_mid,cuisine_cat,duplicate
0,0,1-V:U,"The Outpost Hotel Sentosa, 10 Artillery Avenue...",$31 - $50,Asian Variety,3.5,1.252299,103.820211,40.5,0,0
1,1,10 At Claymore,"Pan Pacific Orchard, 10 Claymore Road Level 2",$51 - $80,Multi-Cuisine,4.0,1.307401,103.829904,65.5,1,0
3,3,100g Korean BBQ,93 Amoy Street,$21 - $30,Korean,3.5,1.281299,103.847092,25.5,2,0
4,4,10th Chocolate Street,"GSM Buidling, 141 Middle Road",$11 - $20,Belgian,3.5,1.299438,103.852403,15.5,3,0
5,5,123 Seafood,"Chinatown Complex Market and Food Centre, 335 ...",$0 - $9,"Singaporean, Chinese",3.5,1.282360,103.843546,4.5,4,1


### Save into a CSV

In [24]:
projectD.to_csv("cleaned_data.csv")
dupedD.to_csv("duped_data.csv")

In [32]:
cuisines = pd.Series(cuisines)
cuisines.to_csv("cuisines.csv", index=False)